In [1]:
#Importing the necessary libraries
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

C:\Users\binur\anaconda3\envs\tfradeon\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Loading the data
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]],
                   dtype='float32')

targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

* Input and target matrices are loaded as NumPy arrays which is should be converted to torch tensors using the torch.from_numpy() method.

In [3]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [4]:
print(inputs)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])


In [5]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])


* **TensorDataset** wraps inputs and targets tensors into a single dataset. We can access rows from the dataset as tuples.

In [6]:
dataset = TensorDataset(inputs,targets)

In [8]:
dataset[:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

* Using **DataLoader** we can convert the dataset into batches of predefined batch size and create batches by picking samples from the dataset randomly.

In [9]:
batch_size = 3
train_loader = DataLoader(dataset,batch_size=batch_size,shuffle=True)

* We can access the data from DataLoader as a tuple pair containing input and corresponding targets using a for loop which enables us to load batches directly into a training loop.

In [10]:
for inp, target in train_loader:
    print(inp)
    print(target)
    break

tensor([[101.,  44.,  37.],
        [ 87., 135.,  57.],
        [ 92.,  87.,  64.]])
tensor([[ 21.,  38.],
        [118., 134.],
        [ 82., 100.]])


* **torch.randn** generates tensors randomly from a uniform distribution with mean 0 and standard deviation 1.

In [11]:
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)
print(w)
print(b)

tensor([[-0.4490, -0.1964,  0.4375],
        [-0.6483,  0.3561,  1.0225]], requires_grad=True)
tensor([ 0.9262, -0.4477], requires_grad=True)


* The model is just a mathematical equation establishing a linear relationship between weights and outputs. Matrix multiplication is performed (**@ represents matrix multiplication**) with the input batch and the transpose of the weights.

In [12]:
def model(x):
    return x @ w.t() + b

In [14]:
for x,y in train_loader:
    preds = model(x)
    print('Prediction is :n',preds)
    print('Actual targets is :',y)
    break

Prediction is :n tensor([[-17.3975,  62.2501],
        [-26.4495,  19.0485],
        [-18.2839,  60.5793]], grad_fn=<AddBackward0>)
Actual targets is : tensor([[104., 118.],
        [ 57.,  69.],
        [103., 119.]])


* **numel()** method returns the number of elements in the tensor.

In [15]:
def mse_loss(predictions,targets):
    difference = predictions - targets
    return torch.sum(difference * difference) / difference.numel()

In [17]:
for x,y in train_loader:
    preds = model(x)
    print('Prediction is :',preds)
    print('Actual target is :',y)
    print('Loss is:',mse_loss(preds,y))
    break

Prediction is : tensor([[-18.2839,  60.5793],
        [-18.0313,  61.5837],
        [-29.2159,  37.3337]], grad_fn=<AddBackward0>)
Actual target is : tensor([[103., 119.],
        [102., 120.],
        [ 81., 101.]])
Loss is: tensor(8690.6143, grad_fn=<DivBackward0>)


* Gradient descent is an optimization algorithm that calculates the derivative/gradient of the loss function to update the weights and correspondingly reduce the loss or find the minima of the loss function.


* Steps to implement Gradient Descent in PyTorch,
1. First, calculate the loss function
2. Find the Gradient of the loss with respect to independent variables
3. Update the weights and bais
4. Repeat the above step
5. Now let’s get into coding and implement Gradient Descent for 50 epochs,

In [19]:
epochs = 50
for i in range(epochs):
    #Iterate through training dataloader
    for x,y in train_loader:
        #Generate prediction
        preds = model(x)
        
        #Gets the loss and perform backpropagation
        loss = mse_loss(preds,y)
        loss.backward()
        
        #Lets update the weights
        with torch.no_grad():
            w -= w.grad * 1e-6
            b -= b.grad * 1e-6
            
            #Set the gradients to zero
            w.grad.zero_()
            b.grad.zero_()
            
        print(f'Epoch {i}/{epochs}: Loss: {loss}')

Epoch 0/50: Loss: 9316.3896484375
Epoch 0/50: Loss: 9681.3037109375
Epoch 0/50: Loss: 6102.611328125
Epoch 0/50: Loss: 3368.697021484375
Epoch 0/50: Loss: 7756.03662109375
Epoch 1/50: Loss: 5367.17431640625
Epoch 1/50: Loss: 6045.43115234375
Epoch 1/50: Loss: 10409.9111328125
Epoch 1/50: Loss: 4623.92333984375
Epoch 1/50: Loss: 3719.714111328125
Epoch 2/50: Loss: 5218.77392578125
Epoch 2/50: Loss: 6909.79296875
Epoch 2/50: Loss: 4424.78759765625
Epoch 2/50: Loss: 4729.55322265625
Epoch 2/50: Loss: 3978.628173828125
Epoch 3/50: Loss: 5384.90185546875
Epoch 3/50: Loss: 5436.982421875
Epoch 3/50: Loss: 3341.990966796875
Epoch 3/50: Loss: 5664.88134765625
Epoch 3/50: Loss: 1401.2091064453125
Epoch 4/50: Loss: 1955.2626953125
Epoch 4/50: Loss: 4124.0224609375
Epoch 4/50: Loss: 2381.63623046875
Epoch 4/50: Loss: 4463.767578125
Epoch 4/50: Loss: 4902.69677734375
Epoch 5/50: Loss: 1761.9635009765625
Epoch 5/50: Loss: 4095.253173828125
Epoch 5/50: Loss: 3874.088623046875
Epoch 5/50: Loss: 1636.

In [21]:
for x,y in train_loader:
    preds = model(x)
    print('Prediction is :',preds)
    print('Actual targets is :',y)
    break

Prediction is : tensor([[ 94.2435, 124.2491],
        [ 45.4435,  35.8608],
        [ 60.4863,  69.4806]], grad_fn=<AddBackward0>)
Actual targets is : tensor([[104., 118.],
        [ 22.,  37.],
        [ 56.,  70.]])
